In [9]:
import pandas as pd
import json
from openai import OpenAI
from fpdf import FPDF 
import os
import requests
from datetime import datetime, timedelta
import pytz
import unicodedata
from secret_like import OPENAI_API_KEY # Remplacer par un fichier secret.py avec la clé api dedans

In [10]:
df_ligne = pd.read_csv("referentiel-des-lignes.csv", delimiter=";")
df_ligne = df_ligne[['ID_Line', 'Name_Line']]
df_ligne.head()

,ID_Line,Name_Line
0,C00003,Licorne
1,C00004,483
2,C00011,3105
3,C00017,3118
4,C00025,3146


In [11]:
# Fonction pour appeler l'API SIRI
def call_siri_api():
    """
    Appel à l'API SIRI pour récupérer les messages généraux
    """
    url = "https://prim.iledefrance-mobilites.fr/marketplace/general-message?LineRef=ALL"

    TOKEN = "HSv2mYkd7MbZ2rkiVTqE3Gh3AAiMh9SA"
    
    try:
        # Headers - ajoutez ici vos clés d'API si nécessaires
        headers = {
            "apiKey": TOKEN,  # Décommentez et ajoutez votre clé si nécessaire
            "Content-Type": "application/json",
            "Accept": "application/json",
        }
        
        response = requests.get(url, headers=headers)
        
        # Vérifier si la requête a réussi
        if response.status_code == 200:
            return response.text
        else:
            print(f"Erreur lors de l'appel API: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Exception lors de l'appel API: {e}")
        return None

In [12]:
# Fonction pour extraire les données du message SIRI avec filtre sur les 15 dernières minutes
def extract_siri_data(json_data):
    """
    Extrait les informations pertinentes du message SIRI et les formate dans un dataframe
    Filtre seulement les messages créés dans les 15 dernières minutes
    """
    # Charger les données JSON
    data = json.loads(json_data)
    
    # Vérifier si les données ont le format attendu
    if not data.get('Siri') or not data['Siri'].get('ServiceDelivery') or not data['Siri']['ServiceDelivery'].get('GeneralMessageDelivery'):
        print("Le format des données SIRI n'est pas celui attendu.")
        return pd.DataFrame()
    
    # Liste pour stocker les données extraites
    rows = []
    
    # Calculer le timestamp d'il y a 15 minutes
    now = datetime.now(pytz.timezone('Europe/Paris'))
    fifteen_minutes_ago = now - timedelta(minutes=15)

    print("Heure actuel :", now)
    print('15 min :', fifteen_minutes_ago)
    
    # Parcourir les messages
    for message_delivery in data['Siri']['ServiceDelivery']['GeneralMessageDelivery']:
        if 'InfoMessage' in message_delivery:
            for info in message_delivery['InfoMessage']:
                # Extraire le timestamp d'enregistrement
                recorded_time_str = info.get('RecordedAtTime', '')
                
                # Vérifier si le message est récent (moins de 15 minutes)
                if recorded_time_str:
                    try:
                        # Convertir le timestamp ISO en objet datetime
                        recorded_time = datetime.fromisoformat(recorded_time_str.replace('Z', '+00:00'))
                        
                        # Skip si le message est plus ancien que 15 minutes
                        if recorded_time < fifteen_minutes_ago:
                            continue
                            
                    except (ValueError, TypeError) as e:
                        print(f"Erreur lors du parsing de la date: {e}, date: {recorded_time_str}")
                        continue
                
                # Extraire les informations nécessaires
                valid_until = info.get('ValidUntilTime', '')
                
                # Extraire la ligne concernée (si disponible)
                ligne = "Non spécifiée"
                if 'Content' in info and 'LineRef' in info['Content'] and info['Content']['LineRef']:
                    line_ref = info['Content']['LineRef'][0].get('value', '')
                    # Extraire le numéro de ligne, exemple: "STIF:Line::C01073:" -> "1073"
                    if 'STIF:Line::C' in line_ref:
                        ligne_num = line_ref.split('STIF:Line::C')[1].strip(':')
                        # Convertir en nombre et enlever les zéros en tête
                        try:
                            ligne = str(int(ligne_num))
                        except ValueError:
                            ligne = ligne_num
                            
                # Catégorie (type de message)
                categorie = info.get('InfoChannelRef', {}).get('value')
                
                # Texte du message
                texte = ""
                if 'Content' in info and 'Message' in info['Content'] and info['Content']['Message']:
                    message_text = info['Content']['Message'][0].get('MessageText', {})
                    if message_text:
                        texte = message_text.get('value', '')
                
                # Convertir les dates ISO en format plus lisible
                try:
                    date_debut = datetime.fromisoformat(recorded_time_str.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M')
                except (ValueError, TypeError):
                    date_debut = recorded_time_str
                
                try:
                    date_fin = datetime.fromisoformat(valid_until.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M')
                except (ValueError, TypeError):
                    date_fin = valid_until
                
                # Ajouter à notre liste de données
                rows.append({
                    "Lignes": ligne,
                    "Date de début": date_debut,
                    "Date de fin": date_fin,
                    "Catégorie": categorie,
                    "Texte de l'ICV gare et bord": texte
                })
    
    # Créer un dataframe avec nos données
    df = pd.DataFrame(rows)
    if df.empty:
        print("Aucun message créé dans les 15 dernières minutes n'a été trouvé.")
    else:
        print(f"Nombre de messages récents trouvés: {len(df)}")
    
    return df

In [23]:
# Code complet d'intégration avec le code précédent
def process_siri_to_pdf(output_pdf="Synthese_Perturbations_SIRI.pdf"):
    """
    Appelle l'API SIRI, traite les données et génère un PDF de synthèse
    des messages des 15 dernières minutes
    """
    # 1. Appeler l'API SIRI
    print("Appel de l'API SIRI...")
    siri_json_data = call_siri_api()
    
    if not siri_json_data:
        print("Échec de l'appel API.")
        return
    
    print("Données reçues de l'API SIRI.")
    
    # 2. Extraire les données du JSON SIRI (uniquement les 15 dernières minutes)
    df = extract_siri_data(siri_json_data)

    # Merge des deux dataframes sur les IDs
    final_df = df.merge(df_ligne, how='left', left_on='Lignes', right_on='ID_Line')

    # Facultatif : renommer la colonne pour plus de clarté
    final_df.rename(columns={'Name_Line': 'Nom de la ligne'}, inplace=True)

    # Supprimer l’ID en double si tu veux
    final_df.drop(columns=['ID_Line'], inplace=True)
    
    if df.empty:
        print("Aucune donnée pertinente n'a été extraite du message SIRI (15 dernières minutes).")
        return
        
    print(f"Données extraites: {len(df)} messages des 15 dernières minutes")
    print(df)
    
    # 3. Générer un prompt pour OpenAI
    def generate_prompt(dataframe):
        lignes = dataframe['Lignes'].unique()

        prompt = """Tu es un assistant de la SNCF. À partir des données suivantes extraites de messages SIRI des 15 dernières minutes, rédige une synthèse claire, concise et structurée à destination du directeur opérationnel de ligne.

Utilise le format suivant comme exemple :

Ligne C :
• Vacances Noël : Offre de transport réduite [...] 
• Accidents de personne : [...] 

Ligne T11 :
• Mouvement social local : [...] 
• Incident de signalisation : [...] 

Ensuite, rédige une section **Analyse des impacts inter-lignes** au format suivant :

**Ligne impactée** : courte explication de l’impact (nature, cause, gares concernées)

Exemples :
• Ligne D : Risque de saturation en gare de Paris Nord suite à un incident sur la ligne B, interconnectée à Châtelet-les-Halles.
• Ligne E : Possibles reports de voyageurs venant de la ligne P, affectée par un mouvement social à Meaux.

N'invente rien. Tu peux déduire un impact uniquement si les données le justifient clairement (gares communes, interconnexions, afflux attendus, etc).

Voici les données brutes à utiliser :\n\n"""

        for ligne in lignes:
            sous_df = dataframe[dataframe['Lignes'] == ligne]
            for _, row in sous_df.iterrows():
                texte = row["Texte de l'ICV gare et bord"].strip().replace('\n', ' ')
                prompt += (
                    f"Ligne {row['Lignes']} | {row['Catégorie']} | "
                    f"{row['Date de début']} - {row['Date de fin']} | {texte} \n"
                )
    
        prompt += (
            "\nCommence par la synthèse ligne par ligne, puis rédige une section "
            "**Analyse des impacts inter-lignes** au format demandé ci-dessus.\n"
        )
        
        return prompt





    prompt_text = generate_prompt(df)
    print("Prompt généré pour OpenAI")
    
    # 4. Appel à l'API OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Tu es un assistant expert en rédaction opérationnelle pour la SNCF."},
            {"role": "user", "content": prompt_text}
        ],
        temperature=0.4,
        max_tokens=1200
    )

    synthese_text = response.choices[0].message.content
    print("Synthèse générée par OpenAI")
    
    # 5. Génération du PDF
    def enregistrer_pdf_alternative(nom_fichier, contenu_texte):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font('Arial', '', 12)

        current_time = datetime.now().strftime("%d/%m/%Y %H:%M")
        pdf.cell(0, 10, f"Synthèse des perturbations réseau SNCF ({current_time})", 0, 1, 'C')
        pdf.cell(0, 10, "Messages des 15 dernières minutes", 0, 1, 'C')
        pdf.ln(5)

        # Remplacer les caractères problématiques
        replacements = {
            '\u2022': '-', '\u2019': "'", '\u2018': "'",
            '\u201c': '"', '\u201d': '"', '\u2013': '-', '\u2014': '--', '\u2026': '...',
            '\u2006': ' ',  # espace fine
        }

        for old, new in replacements.items():
            contenu_texte = contenu_texte.replace(old, new)

        # Supprimer ou normaliser les caractères non Latin-1 restants
        def to_latin1(text):
            return unicodedata.normalize('NFKD', text).encode('latin-1', 'ignore').decode('latin-1')

        contenu_texte = to_latin1(contenu_texte)

        for ligne_contenu in contenu_texte.split('\n'):
            pdf.multi_cell(0, 10, ligne_contenu)

        pdf.output(nom_fichier)

    # Générer le PDF
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        output_pdf_name = f"Synthese_Perturbations_{timestamp}.pdf"
        enregistrer_pdf_alternative(output_pdf_name, synthese_text)
        print(f"PDF généré avec succès: {output_pdf_name}")
    except Exception as e:
        print(f"Erreur lors de la génération du PDF: {e}")
        
    return output_pdf_name

In [25]:
# Pour exécuter le processus complet
if __name__ == "__main__":
    try:
        pdf_file = process_siri_to_pdf()
        print(f"Traitement terminé. Fichier PDF: {pdf_file}")
    except Exception as e:
        print(f"Erreur lors du traitement: {e}")

Appel de l'API SIRI...
Données reçues de l'API SIRI.
Heure actuel : 2025-04-07 16:05:14.548438+02:00
15 min : 2025-04-07 15:50:14.548438+02:00
Nombre de messages récents trouvés: 9
Données extraites: 9 messages des 15 dernières minutes
          Lignes     Date de début       Date de fin     Catégorie  \
0           1365  2025-04-07 13:51  2025-04-14 03:14  Perturbation   
1           1227  2025-04-07 13:59  2025-04-19 01:59  Perturbation   
2  Non spécifiée  2025-04-07 14:04  2025-04-07 14:14  Perturbation   
3           1374  2025-04-07 13:51  2025-04-08 02:30  Perturbation   
4           1365  2025-04-07 13:52  2025-04-14 03:14  Perturbation   
5           1185  2025-04-07 13:59  2025-04-10 18:59  Perturbation   
6           1186  2025-04-07 14:00  2025-04-10 18:59  Perturbation   
7           1124  2025-04-07 13:58  2025-04-10 18:59  Perturbation   
8           1383  2025-04-07 14:01  2025-04-08 02:30  Perturbation   

                         Texte de l'ICV gare et bord  
0  La li